In [3]:
import pickle
import gensim
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import defaultdict
from preprocess import load

In [4]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [5]:
data = load()

In [24]:
'''
Arguments:
sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, 
sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5,
cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, 
sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=()
'''
sg=1
size=300
window=10
min_count=1
iter=25
model = gensim.models.word2vec.Word2Vec(data['train_tokens'], 
                                        sg=sg, size=size, window=window, min_count=min_count, iter=iter)

model_name = '{0}-sz{1}-win{2}-minc{3}-iter{4}'.format('sg' if sg==1 else 'cbow', size, window, min_count, iter)
model.save('data/w2v-' + model_name + '.model')

In [17]:
model_name = 'w2v-sg0-sz300-win5-minc1-hs0-neg5-iter25-samp0.001'
model = gensim.models.word2vec.Word2Vec.load('data/{}.model'.format(model_name))

In [18]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.vectors[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [19]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model.wv)
train_mean_embedded = mean_embedding_vectorizer.transform(data['train_tokens'])
test_mean_embedded = mean_embedding_vectorizer.transform(data['test_tokens'])

In [20]:
from sklearn.linear_model import LogisticRegression

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    m = LogisticRegression()
    mf = m.fit(train_mean_embedded, train[j])
    preds[:,i] = mf.predict_proba(test_mean_embedded)[:,1]

In [21]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission/submission-toxicw2v-docmean-lr-{}.csv'.format(model_name), index=False)